### Base Operations

In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import time
from IPython.display import display
import copy

In [ ]:
class Lattice():
	''' 
	In triangular lattice there are 6 adjacent lattice points
	Here, every second row in the matrix is pushed forward to create the lattice,
	because of this, the environmnet of them is indexed differnetly,
	while using a matrix description.
	'''
	adj_latPoint1 = np.array([[-1,1], [0,1], [-1,0], [1,0], [-1,-1], [0,-1]])
	adj_latPoint2 = np.array([[0,1], [1,1], [-1,0], [1,0], [0,-1], [1,-1]])

	def __init__(self, N):
		self.N = N
		self.lattice = np.zeros([self.N,self.N,2], dtype=bool)
		self.particles = []
		self.adjM = sp.sparse.lil_matrix((0,0), dtype=bool)
	def __copy__(self):
		newL = type(self)(self.N, self.N_par)
		newL.lattice = copy.copy(self.lattice)
		newL.particles = copy.deepcopy(self.particles)
		newL.adjM = copy.copy(self.adjM)
		return newL

	def addParticle(self, particle): # particle is Particle type
		tmp = self.adjM
		self.adjM = sp.sparse.lil_matrix((len(self.particles)+1,len(self.particles)+1), dtype=bool)
		self.adjM[:len(self.particles), :len(self.particles)] = tmp
		# new particles position in lattice
		self.lattice[*particle.pos] = True
		self.particles.append(particle)
		# adjacency matrix and neighbours
  
		'''NEEDED   Deciding how to check neighbours'''
		for adj in particle.pos + neighbours:
			adj[0] = adj[0]%self.N
			adj[1] = adj[1]%self.N
			if self.lattice[*list(adj)]:
				i = self.getParIndex(list(adj))
				self.adjM[-1, i] = True
				self.adjM[i, -1] = True

	def subParticle(self, index): # index in self.particles list
		self.particles.pop(index)
		self.lattice[*self.particles[index].pos] = False
		ind = list(np.arange(len(self.particles), dtype=int))
		ind.pop(index)
		self.adjM = self.adjM[ind,:]
		self.adjM = self.adjM[:,ind]
		  
	def getParIndex(self, pos):
		for j,p in enumerate(self.particles):
			if np.array_equal(p.pos, pos):
				return j	


	def getRateFactors(self,T):
		# creating n_new by checking the neighbours of the neighbours of each particle
		occ_x,occ_y = np.where(self.lattice == True)
		occ_sites = [[x,y] for x,y in zip(occ_x,occ_y)]
		'''NEEDED   Deciding how to check neighbours'''
		unocc_nextto_occ = [[site for site in neighbours[*occ] if self.lattice[*site] == False] for occ in occ_sites]

		origin = []
		destination = []
		probability = []
		for o,d_collection in zip(occ_sites,unocc_nextto_occ):
			n_old = len(np.where(self.adjM[self.getParIndex(o)] == True))
			for d in d_collection:
				n_new = sum([1 for site in neighbours[*d] if self.lattice[*site] == True])
				origin += [o]
				destination += [d]
				probability += [self.particles[o].E/T * np.exp(-(n_old-n_new)/T)]
		return origin, destination, probability




		# adj_adjM = np.ndarray([self.N_par,6])
		# for i,p in enumerate(self.particles):
		# 	if p.pos[1]%2 == 0:
		# 		adj_latPoint = self.adj_latPoint1
		# 	else:
		# 		adj_latPoint = self.adj_latPoint2

		# 	for j,adj in enumerate(p.pos+adj_latPoint): # going around the particle's adjacents lattice points
		# 		# boundary conditions: periodic in X and Y
		# 		if adj[1] < 0:
		# 			adj[1] = self.N-1
		# 		if adj[1] >= self.N:
		# 			adj[1] = 0
		# 		if adj[0] < 0:
		# 			adj[0] = self.N-1
		# 		if adj[0] >= self.N:
		# 			adj[0] = 0
				
		# 		if not self.lattice[adj[0], adj[1]]: # if an empty lattice point
		# 			if adj[1]%2 == 0:
		# 				adjadj_latPoint = self.adj_latPoint1
		# 			else:
		# 				adjadj_latPoint = self.adj_latPoint2

		# 			neighbour = 0
		# 			for adjadj in (adj+adjadj_latPoint): # going around the adjacent lattice points' neighbours
		# 				# boundary conditions: periodic in X and Y
		# 				if adjadj[1] < 0:
		# 					adjadj[1] = self.N-1
		# 				if adjadj[1] >= self.N:
		# 					adjadj[1] = 0
		# 				if adjadj[0] < 0:
		# 					adjadj[0] = self.N-1
		# 				if adjadj[0] >= self.N:
		# 					adjadj[0] = 0

		# 				if self.lattice[adjadj[0], adjadj[1]]:
		# 					neighbour += 1
		# 			adj_adjM[i,j] = neighbour-1 # minus itself
		# 		else:
		# 			adj_adjM[i,j] = -np.Inf
		# return adj_adjM
	
	def updateLattice(self, par, move):
		# changing the variables, attributes after a hopping
		if self.particles[par].pos[1]%2 == 0:
			adj_latPoint = self.adj_latPoint1
		else:
			adj_latPoint = self.adj_latPoint2
		# update lattice point
		self.lattice[self.particles[par].pos[0], self.particles[par].pos[1]] = False
		new_pos = self.particles[par].pos + adj_latPoint[move]
		if new_pos[1] < 0:
			new_pos[1] = self.N-1
		if new_pos[1] >= self.N:
			new_pos[1] = 0
		if new_pos[0] < 0:
			new_pos[0] = self.N-1
		if new_pos[0] >= self.N:
			new_pos[0] = 0
		self.lattice[new_pos[0], new_pos[1]] = True
		# update particle position
		self.particles[par].pos = new_pos
		# update adjacency matrix
		self.adjM[:,par] = False
		self.adjM[par,:] = False
		if self.particles[par].pos[1]%2 == 0:
			adj_latPoint = self.adj_latPoint1
		else:
			adj_latPoint = self.adj_latPoint2
		for adj in (self.particles[par].pos+adj_latPoint):
			if adj[1] < 0:
				adj[1] = self.N-1
			if adj[1] >= self.N:
				adj[1] = 0
			if adj[0] < 0:
				adj[0] = self.N-1
			if adj[0] >= self.N:
				adj[0] = 0
			
			if self.lattice[adj[0], adj[1]]:
				for j,p_adj in enumerate(self.particles):
					if np.array_equal(p_adj.pos, adj):
						break
				self.adjM[par,j] = True
				self.adjM[j,par] = True

	def neighbourCorelation(self):
		cor = np.zeros([self.N_par])
		for i,p in enumerate(self.particles):
			if p.pos[1]%2 == 0:
				adj_latPoint = self.adj_latPoint1
			else:
				adj_latPoint = self.adj_latPoint2

			for adj in (p.pos+adj_latPoint):
				# boundary conditions: periodic in X and Y
				if adj[1] < 0:
					adj[1] = self.N-1
				if adj[1] >= self.N:
					adj[1] = 0
				if adj[0] < 0:
					adj[0] = self.N-1
				if adj[0] >= self.N:
					adj[0] = 0
				# corelation between neighbouring pairs
				cor[i] += self.lattice[p.pos[0],p.pos[1]] * self.lattice[adj[0],adj[1]]
		return cor.sum()/self.N_par

	def doAHop(self, T):
		# physics behind the scene
		def randomEvent(rates):
			r = np.random.rand() * rates.sum()

			for i in range(rates.shape[0]):
				for j in range(rates.shape[1]):
					r = r - rates[i,j]
					if r < 0:
						return [i,j]
					
		n_old = self.adjM.sum(0)
		n_new = self.checkAdjacentsNeighbours()
		hopping_rate = np.exp(-(n_old.T - n_new)/T)
		self.updateLattice(*randomEvent(hopping_rate))

	def hoppingSequence(self, N_hop=1, T=1.0):
		# repeated hopping sequence and displaying
		# hdisplay = display("", display_id=True)
		# fig, ax = plt.subplots()
		# xv, yv = np.meshgrid(np.arange(self.N, dtype=float), np.arange(self.N, dtype=float))
		# xv[1:self.N:2, :] += 0.5
		# # the initial state
		# xl = []; yl = []
		# for i,p in enumerate(self.particles):
		#     if p.pos[1]%2 == 1:
		#         xl.append(p.pos[0]+0.5)
		#     else:
		#         xl.append(p.pos[0])
		#     yl.append(p.pos[1])
		#     # ax.annotate(str(i), (xl[-1], yl[-1])) 
		# ax.scatter(xv, yv, c='b')
		# ax.scatter(xl, yl, c='r')
		# hdisplay.update(fig)
		# time.sleep(1)

		cor = []
		cor.append(self.neighbourCorelation())
		if type(T) != float: # hops for varying T
			ran = len(T)
		else: # for fixed number of hops at a certain T
			ran = N_hop
			T = T*np.ones([N_hop])
		for frame in range(ran):
			self.doAHop(T[frame])
			cor.append(self.neighbourCorelation())
			# xl = []; yl = []
			# ax.clear()
			# for i,p in enumerate(self.particles):
			#     if p.pos[1]%2 == 1:
			#         xl.append(p.pos[0]+0.5)
			#     else:
			#         xl.append(p.pos[0])
			#     yl.append(p.pos[1])
				# ax.annotate(str(i), (xl[-1], yl[-1]))
			# ax.scatter(xv, yv, c='b')
			# ax.scatter(xl, yl, c='r')
			# hdisplay.update(fig)
			# time.sleep(0.3)

		fig, ax = plt.subplots()
		xv, yv = np.meshgrid(np.arange(self.N, dtype=float), np.arange(self.N, dtype=float))
		xv[1:self.N:2, :] += 0.5
		# the initial state
		xl = []; yl = []
		for i,p in enumerate(self.particles):
			if p.pos[1]%2 == 1:
				xl.append(p.pos[0]+0.5)
			else:
				xl.append(p.pos[0])
			yl.append(p.pos[1])
			# ax.annotate(str(i), (xl[-1], yl[-1])) 
		ax.scatter(xv, yv, c='b')
		ax.scatter(xl, yl, c='r')
		plt.show()
		# plt.close()
		plt.plot(np.arange(ran+1), cor)
		plt.xlabel("Hopping numbers")
		plt.ylabel("Corelation per particle")
		plt.show()

	def plot(self):
		xv, yv = np.meshgrid(np.arange(self.N, dtype=float), np.arange(self.N, dtype=float))
		xv[1:self.N:2, :] += 0.5
		xl = []; yl = []
		for i,p in enumerate(self.particles):
			if p.pos[1]%2 == 1:
				xl.append(p.pos[0]+0.5)
			else:
				xl.append(p.pos[0])
			yl.append(p.pos[1])
			# plt.annotate(str(i), (xl[-1], yl[-1]))
		plt.scatter(xv, yv, c='b')
		plt.scatter(xl, yl, c='r')
		plt.show()


class Particle():
	def __init__(self, pos, E):
		self.pos = pos # should be a list
		self.E = E
		
	def __copy__(self):
		return type(self)(self.pos, self.E)

In [25]:
# TESTING SITE
a = sp.sparse.lil_matrix((5,5))
x, y = np.meshgrid(np.arange(5), np.arange(5))
for i in range(5):
    for j in range(5):
        a[i,j] = x[i,j]*y[i,j]
print(a)
print(a.shape)

r = list(np.arange(5, dtype=int))
r.pop(3)
a = a[r,:]
a = a[:,r]

print(a.shape)
print(a)

  (1, 1)	1.0
  (1, 2)	2.0
  (1, 3)	3.0
  (1, 4)	4.0
  (2, 1)	2.0
  (2, 2)	4.0
  (2, 3)	6.0
  (2, 4)	8.0
  (3, 1)	3.0
  (3, 2)	6.0
  (3, 3)	9.0
  (3, 4)	12.0
  (4, 1)	4.0
  (4, 2)	8.0
  (4, 3)	12.0
  (4, 4)	16.0
(5, 5)
(4, 4)
  (1, 1)	1.0
  (1, 2)	2.0
  (1, 3)	4.0
  (2, 1)	2.0
  (2, 2)	4.0
  (2, 3)	8.0
  (3, 1)	4.0
  (3, 2)	8.0
  (3, 3)	16.0


In [51]:
sites = np.where(np.random.rand(5,5) < 0.5, 0, 1)
print(sites)
[[x, y] for x,y in zip(np.where(sites != 0)[0], np.where(sites != 0)[1])]



[[0 1 1 1 0]
 [0 0 0 1 0]
 [0 1 1 1 1]
 [1 1 1 0 0]
 [1 1 1 0 0]]


[[0, 1],
 [0, 2],
 [0, 3],
 [1, 3],
 [2, 1],
 [2, 2],
 [2, 3],
 [2, 4],
 [3, 0],
 [3, 1],
 [3, 2],
 [4, 0],
 [4, 1],
 [4, 2]]

In [ ]:
def gridFreezing(N, N_par, N_hop, T):
    # input is either fixed T and number of hops
    # or a T array with arbitrary N_hops
    lattice = TriangleLattice(N, N_par)
    lattice.fillParticle()
    lattice.plot()
    lattice_var = copy.copy(lattice)

    # comparisons
    print("Varying T while iterating")
    lattice.hoppingSequence(N_hop, T)
    # print("Steady T while iterating")
    # lattice_var.hoppingSequence(N_hop)

N = 40 # lattice grid size, even number
N_par = 400 # particles on the grid
N_hop = 1000 # number of hoppings to occur
T = 1/(np.linspace(0.1, 10, N_hop)) # freezing in terms of rates (rates increase for high neighbours)

gridFreezing(N, N_par, N_hop, T)